In [2]:
import os
import numpy as np
import pandas as pd

In [3]:
DATA_TEXT_PATH = "/content/drive/MyDrive/MARATHI_WSD_Data/Word_Net/data_txt.TXT"
IDX_NOUN_PATH = "/content/drive/MyDrive/MARATHI_WSD_Data/Word_Net/idxnoun_txt.TXT"
IDX_PATH = "/content/drive/MyDrive/MARATHI_WSD_Data/Word_Net/index_txt.TXT"

In [4]:
opened_file = open(IDX_NOUN_PATH, "r", encoding="utf-8-sig")
idx_noun_content = opened_file.read()

In [5]:
noun_content = idx_noun_content.split("\n")

In [6]:
nouns = []
for noun in noun_content:
    text = noun.split(" ")
    try:
        skip1 = int(text[2])
        skip2 = skip1+1
        nouns.append((text[0], text[2+skip2+1:]))
    except:
        pass

In [7]:
nouns[:5]

[('-तरंग', ['00005942']),
 ('-धारा', ['00027382']),
 ('-सारखा', ['00026718']),
 ('1000000', ['00031077']),
 ('10000000', ['00015838'])]

In [8]:
opened_file = open(DATA_TEXT_PATH, "r", encoding="utf-8-sig")
data_content = opened_file.read()

In [9]:
data_content = data_content.split("\n")

In [10]:
all_syns = []
for syn in data_content:
    try:
        ids, sentence = syn.split(" | ")
        x = ids.split(" ")
        all_syns.append((x[0], x[3].split(":"), sentence))
    except:
        pass

In [11]:
all_syns[:5]

[('00000001', ['अजन्मा'], 'ज्यास जन्म नाही असा:"ईश्वर अजन्मा आहे"'),
 ('00000002',
  ['अशुभ', 'अमंगळ'],
  'शुभ नाही असा:"या योगामूळे कुंडलीतील इतर अशुभ योगांचा नाश होतो."'),
 ('00000003',
  ['अप्रविष्ट'],
  'ज्याने प्रवेश केला नाही असा:"अप्रविष्ट व्यक्तींना ताबडतोब आत प्रवेश करू द्या."'),
 ('00000004',
  ['पुण्यभूमी', 'पवित्रभूमी', 'पुण्यस्थान', 'पवित्रस्थान', 'पावनस्थान'],
  'पवित्र मानले गेलेले स्थान:"हिंदूंसाठी काशी ही पुण्यभूमी आहे."'),
 ('00000005',
  ['शिवालय', 'शिवमंदिर'],
  'जिथे शंकराच्या पिंडीची स्थापना केली असून त्याची पूजा होते ती जागा:"ती दर सोमवारी शिवालयात जाते."')]

In [12]:
for i in all_syns:
    if i[0] == "00010818":
        print(i)

('00010818', ['राजस्थान'], 'भारतातील पश्चिमेला असलेला प्रदेश:"राजस्थानाचा बहूतांश भाग वाळवंटी आहे."')


In [13]:
meanings = {}
for syn in all_syns:
    meanings[syn[0]] = syn[2]
len(meanings)

28530

In [14]:
meanings["00010818"]

'भारतातील पश्चिमेला असलेला प्रदेश:"राजस्थानाचा बहूतांश भाग वाळवंटी आहे."'

In [15]:
import pickle

In [16]:
pickle.dump(meanings, open("meanings.pkl", "wb+"))

In [17]:
# # Stemmers Loader
# #Stemmers.txt file contains bigger words at stat so as to remove them first
# stemmers = []

# f = open('marathi_stemmers.txt',encoding='utf8',mode='r')
# stemmers = f.readlines()

# stemmers = [i[:-1] for i in stemmers]

# def stemmer(word):
#     import re
#     for i in stemmers:
#         if (i in word) & (len(word) > (len(i)+1)):
#             return re.sub(i,"",word)
#         else:
#             pass
#     return word

In [18]:
# def levenshteinDistance(s1, s2):
#     if len(s1) > len(s2):
#         s1, s2 = s2, s1

#     distances = range(len(s1) + 1)
#     for i2, c2 in enumerate(s2):
#         distances_ = [i2+1]
#         for i1, c1 in enumerate(s1):
#             if c1 == c2:
#                 distances_.append(distances[i1])
#             else:
#                 distances_.append(1 + min((distances[i1], distances[i1 + 1], distances_[-1])))
#         distances = distances_
#     return distances[-1]

In [19]:
# all_syns = pickle.dump(all_syns)

In [20]:
import difflib

In [21]:
nouns_data = []
for noun in nouns:
    for syn_id in noun[1]:
        for syn in all_syns:
            if syn_id == syn[0]:
                sentence = syn[2].split(" ")
                flag = False
                for synonym in syn[1]:
                    match = difflib.get_close_matches(synonym, sentence)
                    if len(match)!=0:
                        flag = True
                        sentence[sentence.index(match[0])] = noun[0]
                        break
                if flag:
                    nouns_data.append((' '.join(sentence), noun[0], syn_id))

In [22]:
for i in nouns_data:
    if i[2] == "00010818":
        print(i)

In [23]:
for i in "अनामी:अनामी_भाषा:nआन्नाम:आन्नाम_भाषा".split(":"):
    print(i, difflib.get_close_matches(i, 'अनाम ह्या प्रांताच्या लोकांची भाषा:"अनामी ही मोन-ख्मेर कुळातली भाषा आहे."'.split(" ")))

अनामी ['अनाम', 'भाषा:"अनामी']
अनामी_भाषा []
nआन्नाम []
आन्नाम_भाषा []


In [24]:
len(nouns_data)

31436

In [25]:
for i in range(5,10):
    print(nouns_data[i])

('इंग्रजी महिन्यातील बाराव्या दिवशी येणारी तारीख:"पुढच्या महिन्याच्या 14 दसरा आहे."', '14', '00026902')
('शंभर आणि पन्नास मिळून येणारी संख्या:"पंच्याहत्तर आणि पंच्याहत्तर मिळूनदेखील 150 होतात."', '150', '00030156')
('दहा अधिक सात मिळून होणारी संख्या:"नऊ आणि आठची बेरीज 17 होते."', '17', '00013732')
('दहा अधिक आठ मिळून होणारी संख्या:"आपल्या पुराणांची संख्या 18 आहे"', '18', '00002573')
('वीस आणि नऊ मिळून होणारी संख्या:"तीसामधून एक वजा केले असता 29 उत्तर येते."', '29', '00013759')


In [26]:
cnt = 1
for i in nouns_data:
    if ":" not in i[0]:
        cnt+=1
        print(i)
    if cnt == 20:
        break

('ऐंशी आणि तीन मिळून येणारी 83 तीनने भागता येत नाही."', '83', '00015768')
('लांबीचे एक अँगस्ट्रॉम ह्या एककाचे नाम स्वीडिश शास्त्रज्ञाच्या नावावरून दिले आहे."', 'अँगस्ट्रॉम', '00027153')
('मादागास्करची अँटाननरिव्हो हे मादागास्करचे सर्वात मोठे शहर आहे."', 'अँटाननरिव्हो', '00024174')
('लक्ष्मणच्या दोन पुत्रांपैकी अंगद आणि चंद्रकेतु हे दोघे सख्खे भाऊ होते."', 'अंगद', '00019351')
('शरीराची अंगबळ जोरावर तो कोणतेही काम करवून घेऊ शकतो"', 'अंगबळ', '00009673')
('एक अंगिरावृत वर्णन पुराणांमध्ये आढळते."', 'अंगिरावृत', '00029367')
('बोटाला सुई टोचू नये म्हणून व सुईस मागून नेट मिळावा म्हणून तर्जनीत घालण्याचे एक धातूचे अंगुलीत्राण घालून शिवणकामाचा सराव कर"', 'अंगुलीत्राण', '00002010')
('बोटाला सुई टोचू नये म्हणून व सुईस मागून नेट मिळावा म्हणून तर्जनीत घालण्याचे एक धातूचे अंगुष्ठान घालून शिवणकामाचा सराव कर"', 'अंगुष्ठान', '00002010')
('बोटाला सुई टोचू नये म्हणून व सुईस मागून नेट मिळावा म्हणून तर्जनीत घालण्याचे एक धातूचे अंगुस्तन घालून शिवणकामाचा सराव कर"', 'अंगुस्तन', '00002010')
('बोटाला सुई टोचू नये

In [27]:
cleaned_noun_data = []
for i in nouns_data:
    try:
        if len(i[0].split()) != 1:
            if "::" in i[0]:
                meaning, sentence = i[0].split("::")
            elif ":" in i[0]:
                meaning, sentence = i[0].split(":")
            else:
                meaning, sentence = i[0].split(".")
            sentence = sentence[1:len(sentence)-1]
    except:
        if '"' not in i[0]:
            sentence = i[0]
        else:
            start = i[0].index('"')
            end = len(i[0])
            sentence = i[0][start+1:end-1]
    for sent in sentence.split("/"):
        cleaned_noun_data.append((sent, i[1], i[2]))

In [28]:
len(cleaned_noun_data)

32319

In [29]:
cleaned_noun_data[10000:10005]

[('तो संन्यासी अंगावरच्या भगव्या छाटी काहीही बरोबर बाळगत नाही',
  'छाटी',
  '00005252'),
 ('भाला त्याच्या छाताड घुसला ', 'छाताड', '00001960'),
 ('भाला त्याच्या छाती घुसला ', 'छाती', '00001960'),
 ('', 'छाती', '00001962'),
 ('मी साप हाती घेण्याचे छाती करीन पण दुसर्\u200dयाला मारण्याचे नाही',
  'छाती',
  '00002958')]

In [30]:
def consine_similarity(l1, l2):
    c = 0
    for i in range(len(rvector)): 
        c+= l1[i]*l2[i] 
    cosine = c / float((sum(l1)*sum(l2))**0.5)
    return cosine

In [31]:
given_sentence = "मला कार्टून नेटवर्क चॅनेल खूपच आवडते!"
given_noun = "कार्टून"

syn_ids = []
for noun in nouns:
    if noun[0] == given_noun:
        syn_ids.append(noun[1])
print(syn_ids)
cosine_scores = []

sents = []
scores = []
ids = []

for i in syn_ids[0]:
    for j in cleaned_noun_data:
        if i == j[2]:
            x = given_sentence.split(" ")
            y = j[0].split(" ")
            
            X_set = {word for word in x}
            Y_set = {word for word in y}

            # print(X_set, Y_set)

            rvector = X_set.union(Y_set)
            l1, l2 = [], []
            for w in rvector: 
                if w in X_set: 
                    l1.append(1) # create a vector 
                else: 
                    l1.append(0) 
                if w in Y_set: 
                    l2.append(1) 
                else: 
                    l2.append(0)
            cosine_score = consine_similarity(l1,l2)
            sents.append(j[0])
            scores.append(cosine_score)
            ids.append(i)

[['00023951', '00023958']]


In [32]:
print(sents)
print(scores)
print(ids)

['आर. के लक्ष्मण ह्यांचे कार्टून खूप प्रभावी होते.', 'आर. के लक्ष्मण ह्यांचे व्यंगचित्र खूप प्रभावी होते.', 'आर. के लक्ष्मण ह्यांचे व्यंग्यचित्र खूप प्रभावी होते.', 'मुले टीवीवर कार्टून पाहत आहे.']
[0.14433756729740646, 0.0, 0.0, 0.18257418583505536]
['00023951', '00023951', '00023951', '00023958']


In [33]:
final_id = ids[np.argmax(scores)]
print(final_id)

00023958


In [34]:
for syn in all_syns:
    if final_id == syn[0]:
        print("Meaning: ",syn[2].split(":")[0])
        break

Meaning:  कार्टून वा अर्कचित्र वापरून साकारलेली चित्रफीत


In [35]:
meanings = {}
for syn in all_syns:
    meanings[syn[0]] = syn[2]

In [36]:
import pickle

In [37]:
nouns_set = {}
for noun in nouns:
    nouns_set[noun[0]] = noun[1]

cleaned_noun_set = {}
for noun in cleaned_noun_data:
    cleaned_noun_set[noun[-1]] = noun[:-1]

In [38]:
pickle.dump(nouns_set, open("nouns.pkl", "wb+"))
pickle.dump(cleaned_noun_set, open("cleaned_noun_data.pkl", "wb+"))
pickle.dump(meanings, open("meanings.pkl", "wb+"))